<a href="https://colab.research.google.com/github/Anastasia-spl/slim-mom-frontend/blob/master/Lesnov_Petr/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Подключение библиотек для обучение AI

In [2]:
# Основа для модели нейронной сети
from tensorflow.keras.models import Model

# Стандартные слои keras
from tensorflow.keras.layers import Input, Conv3DTranspose, concatenate, Activation, MaxPooling3D, Conv3D, BatchNormalization, Dropout

# Оптимизатор Adam
from tensorflow.keras.optimizers import Adam

# Дополнительные утилиты keras
from tensorflow.keras import utils

# Инструменты для построения графиков
import matplotlib.pyplot as plt

# Инструменты для работы с изображениями
from tensorflow.keras.preprocessing import image

# Инструменты для работы с массивами
import numpy as np

# Системные инструменты
import time, random, gdown, os, gc, shutil

# Дополнительные инструменты визуализации
import seaborn as sns
sns.set_style('darkgrid')

from google.colab import drive
drive.mount('/content/gdrive')

import h5py

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Создание модели

In [3]:
def standardize(image):
  standardize_image = np.zeros(image.shape)

  for z in range(2):
    image_slice = image[:,:,z]
    print(np.mean(image_slice))

    centered = image_slice - np.mean(image_slice)

    if(np.std(centered)!=0):
      centered = centered/np.std(centered)

    standardize_image[:,:,z] = centered

    return standardize_image

In [4]:
def conv_block(input_data, filters, batch_norm):
  x = Conv3D(filters,(3,3,3), padding="same", strides = (1,1,1))(input_data)
  if batch_norm:
    x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Conv3D(filters,(3,3,3), padding="same", strides = (1,1,1))(x)
  if batch_norm:
    x = BatchNormalization()(x)
  x = Activation('relu')(x)

  return x

def Unet_3d(input_img, filters = 8, dropout = 0.2, batch_norm = True):
  c1 = conv_block(input_img, filters, batch_norm)
  p1 = MaxPooling3D(pool_size=(2,2,2), stridez=2)(c1)
  p1 = Dropout(dropout)(p1)

  c2 = conv_block(p1,filters*2,batch_norm)
  p2 = MaxPooling3D(pool_size=(2,2,2), stridez=2)(c2)
  p2 = Dropout(dropout)(p2)

  c3 = conv_block(p2,filters*4,batch_norm)
  p3 = MaxPooling3D(pool_size=(2,2,2), stridez=2)(c3)
  p3 = Dropout(dropout)(p3)

  c4 = conv_block(p3,filters*8,batch_norm)
  p4 = MaxPooling3D(pool_size=(2,2,2), stridez=2)(c4)
  p4 = Dropout(dropout)(p4)

  c5 = conv_block(c4, filters*16, batch_norm)

  u6 = Conv3DTranspose(filters*8, (3,3,3), strides = (2,2,2), padding = "same")(c5)
  u6 = concatenate([u6,c4])
  c6 = conv_block(u6, filters*8, batch_norm)
  c6 = Dropout(dropout)(c6)

  u7 = Conv3DTranspose(filters*4, (3,3,3), strides = (2,2,2), padding = "same")(c6)
  u7 = concatenate([u7,c3])
  c7 = conv_block(u7, filters*4, batch_norm)
  c7 = Dropout(dropout)(c7)

  u8 = Conv3DTranspose(filters*2, (3,3,3), strides = (2,2,2), padding = "same")(c7)
  u8 = concatenate([u8,c2])
  c8 = conv_block(u8, filters*2, batch_norm)
  c8 = Dropout(dropout)(c8)

  u9 = Conv3DTranspose(filters, (3,3,3), strides = (2,2,2), padding = "same")(c8)
  u9 = concatenate([u9,c1])
  c9 = conv_block(u9,filters, batch_norm)

  outputs = Conv3D(2, (1,1,1), activation = 'sigmoid')(c9)

  print(120*'-')
  print(outputs.shape)

  model = Model(inputs=input_img, outputs=outputs)

  return model